In [1]:
from IPython.display import display, HTML
from fasthtml.common import *
from fasthtml.jupyter import *

def create_server(app,rt):
    if IN_NOTEBOOK:
        for port in range(8000,8030):   
            if 'server' in globals():
                print(f"Server already running on port {globals()['server'].port} - stopping it")
                globals()['server'].stop()
            if is_port_free(port):
                server = JupyUvi(app, port=port)

                def HShow(comp, app):
                    @app.get('/')
                    def get(): return comp
                    display(HTML(f'<a href="http://localhost:{port}/" target="_blank">Open in new tab</a>'))
                    return HTMX("/",port=port, iframe_height="300px")
                print(f"Starter server on port {port}")
                Show = partial(HShow, app=app)
                return app, rt, server, HShow, Show

In [2]:
app, rt = fast_app(pico=False, hdrs=[Script(src="https://cdn.tailwindcss.com")])
app, rt, server, HShow, Show = create_server(app, rt)

Starter server on port 8000


In [3]:
def ft_react_raw(code:str='', jsx:bool=False, comp_name:str='default', id:str=None):

    if not id: id = unqid()
    
    babel_cdn = Script(src="https://unpkg.com/@babel/standalone/babel.min.js")

    render_scr = """const root = ReactDOM.createRoot(document.getElementById('%s'));""" % id

    if jsx:
        render_scr += "root.render(<%s />);" % comp_name.title()
    else:
        render_scr += "root.render(React.createElement(%s, null, null));" % comp_name.title()
    
    main_scr = Script(code + render_scr, type="text/babel" if jsx else 'module', data_type="module" if jsx else '')

    if jsx: 
        return (babel_cdn, Div(main_scr, id=id)) 
    else:
        return Div(main_scr, id=id)

In [4]:
def test():
    babel_scr = Script(src="https://unpkg.com/@babel/standalone/babel.min.js")

    id = unqid()

    test_scr = """
    import React from 'https://esm.sh/react';
    import ReactDOM from 'https://esm.sh/react-dom/client';
    import { PieChart, Pie, Sector, Cell, ResponsiveContainer } from 'https://esm.sh/recharts';

    const data = [
      { name: 'Group A', value: 400 },
      { name: 'Group B', value: 300 },
      { name: 'Group C', value: 300 },
      { name: 'Group D', value: 200 },
      { name: 'Group E', value: 278 },
      { name: 'Group F', value: 189 },
    ];

const Chart = () => {
    return (
        <PieChart width={400} height={400}>
          <Pie
            dataKey="value"
            startAngle={180}
            endAngle={0}
            data={data}
            cx="50%"
            cy="50%"
            outerRadius={80}
            fill="#8884d8"
            label
          />
        </PieChart>
      )
    }
"""

    render_str = """const root = ReactDOM.createRoot(document.getElementById('%s'));
    root.render(<Chart />);""" % id

    return (babel_scr, Div(Script(test_scr+render_str, type="text/babel", data_type="module"), id=id))

In [5]:
Show(Main(test(),Hr(), test()))

In [32]:
def SButton(*c, **kwargs):
    id = kwargs.pop("id", unqid())
    
    babel_scr = Script(src="https://unpkg.com/@babel/standalone/babel.min.js")
    
    imports = """
    import {createRoot} from 'https://esm.sh/react-dom/client';
    import {Button} from 'https://esm.sh/shadcn/es2022';
    
    console.log(shad);
    """
    
    render_scr = """const root = createRoot(document.getElementById('%s'));
    root.render(<Button />);""" % id
    
    return (Div(Script(imports+render_scr, type="text/babel", data_type="module"), babel_scr))

In [33]:
Show(SButton())